<a href="https://colab.research.google.com/github/JSJeong-me/AI-Innovation-2024/blob/main/1-3-Torch-DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

In [2]:
# Load the CSV file into a pandas DataFrame
file_path = './winequality-red.csv'
df = pd.read_csv(file_path)

# Convert the DataFrame to a PyTorch tensor
tensor_data = torch.tensor(df.values, dtype=torch.float32)

# Get the shape (dimensions) of the tensor
tensor_shape = tensor_data.shape

tensor_shape

torch.Size([1599, 12])

In [3]:
# Prepare the data
X = tensor_data[:, :-1]  # Features (all columns except the last one)
y = tensor_data[:, -1]    # Target variable ('quality' column)

# Create a dataset and data loader
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [5]:
# Define the neural network model
class WineQualityModel(nn.Module):
    def __init__(self):
        super(WineQualityModel, self).__init__()
        self.fc1 = nn.Linear(11, 64)  # 11 input features
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)   # Output layer for regression

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
# Instantiate the model, define the loss function and the optimizer
model = WineQualityModel()
criterion = nn.MSELoss()  # Mean Squared Error for regression
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs).squeeze()  # Forward pass
        loss = criterion(outputs, targets)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update the weights

    # Print loss every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# The model is now trained and can be used for predictions
model


Epoch [10/100], Loss: 0.3919
Epoch [20/100], Loss: 0.4898
Epoch [30/100], Loss: 0.6699
Epoch [40/100], Loss: 0.2998
Epoch [50/100], Loss: 0.1968
Epoch [60/100], Loss: 0.5728
Epoch [70/100], Loss: 0.5217
Epoch [80/100], Loss: 0.5184
Epoch [90/100], Loss: 0.4859
Epoch [100/100], Loss: 0.4506


WineQualityModel(
  (fc1): Linear(in_features=11, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=1, bias=True)
)

In [8]:
# Select the first 10 samples as test data
test_data = X[:10]
test_targets = y[:10]

# Make predictions using the trained model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    predictions = model(test_data).squeeze()

# Compare predictions with actual values
predictions, test_targets

(tensor([5.2677, 5.3635, 5.3949, 5.7001, 5.2677, 5.3486, 5.2850, 5.3514, 5.2569,
         5.5034]),
 tensor([5., 5., 5., 6., 5., 5., 5., 7., 7., 5.]))

In [9]:
# Define the Mean Absolute Error (MAE) loss function
mae_loss = nn.L1Loss()

# Calculate MAE and MSE for the predictions
mae_value = mae_loss(predictions, test_targets)
mse_value = criterion(predictions, test_targets)  # MSE was already defined as 'criterion'

mae_value.item(), mse_value.item()


(0.6122401356697083, 0.6733812093734741)